<h2>개인 구글 드라이브와 colab 연동</h2>

In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


<h2>"SMSSpamCollection" 데이터를 읽고 문장과 정답을 분리하여 각 리스트에 저장</h2>

<pre>
<b>1. 데이터의 형태(SMSSpamCollection)</b>
  라벨(스팸 또는 햄) \t(tab) 문장 
  
  위와 같은 형태로 저장되어 있음
  
  예시)
    ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    spam\tCustomer service annoncement. You have a New Years delivery waiting for you. Please call 07046744435 now to arrange delivery
    ...
  
  따라서 입력 데이터를 읽고 \t을 기준으로 입력 문장을 분리한 후에 문장과 라벨을 각각 x_data, y_data 리스트에 저장
  
<b>2. 입력 데이터 전체를 사용하지 않고 100개만 추출해서 사용</b>

<b>3. x_data, y_data 형태</b>
  x_data = [ 문장1, 문장2, 문장3, ... 문장100]
  y_data = [ 문장1의 라벨, 문장2의 라벨, 문장3의 라벨, ... 문장100의 라벨]
</pre>

In [2]:
import numpy as np

file_path = "/gdrive/My Drive/colab/svm/SMSSpamCollection.dat"

# 파일 읽기
x_data, y_data = [], [] #x:데이터 y:레이블
with open(file_path,'r',encoding='utf8') as inFile:
  lines = inFile.readlines()
  
lines = lines[:100]  #100개만 읽어옴
  
  #라인 하나씩 가져와서 레이블과 데이터를 탭으로 분리, 뒤쪽엔 문장(sentence), 앞에는 정답레이블(label)
for line in lines:
  line = line.strip().split('\t')
  sentence, label = line[1], line[0]
  x_data.append(sentence)
  y_data.append(label)
  
print("x_data의 개수 : " + str(len(x_data)))
print("y_data의 개수 : " + str(len(y_data)))

x_data의 개수 : 100
y_data의 개수 : 100


<h2>Tokenizer 라이브러리를 사용하여 입력 문장을 index로 치환</h2>

<pre>
<b>1. tokenizer.fit_on_texts(data) 함수를 이용하여 각 단어를 index로 치환하기 위한 딕셔너리 생성</b>
   생성된 딕셔너리는 tokenizer 객체 안에 저장됨
  
  tokenizer.fit_on_texts(data)
  args
    data : 문자열 element를 가지고 있는 리스트
  return
    X
    
  딕셔너리 예시)
    {'to': 1, 'i': 2, 'you': 3, 'a': 4, 'the': 5, 'and': 6, 'for': 7 ... }
    
<b>2. tokenizer.texts_to_sequences(data) 함수를 이용하여 문장 안에 있는 단어들을 index로 치환</b>

  tokenizer.texts_to_sequence(data)
  args
    data : 문자열 element를 가지고 있는 리스트
  return : 
    indexing 된 리스트
    
  indexing 예시)
    x_data indexing 하기 전 : Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    x_data indexing 하기 후 : [38, 93, 239, 240, 241, 242, 53, 11, 243, 72, 94, 244, 245, 126, 246, 247, 73, 74, 248, 127]
    y_data indexing 하기 전 : ham
    y_data indexing 하기 후 : 1
</pre>

In [3]:
# decision tree와 달리 숫자로 매핑해야함
# 데이터 변환 (문자열 -> 숫자)
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer() #스페이스 단위로 끊으는 것(어절단위)

import nltk
nltk.download("stopwords")
stopwords_list=nltk.corpus.stopwords.words("english")

print("영어 불용어 개수",len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words("english"[:10]))

#여기서 x_data와 불용어 비교할 코드 쓰기

fin_xdata=[]

for i in x_data:
  corpus=[] #말뭉치 리스트
  i=i.lower() #불용어들은 다 소문자로 되어있어서 말뭉치 또한 소문자 처리를 해 정확도를 높였다.
  #소문자 처리전: 50%
  #소문자 처리후: 60%
  corpus=i.split() # 말뭉치로 분리

  for stopwords_i in stopwords_list:
    while stopwords_i in corpus:
      corpus.remove(stopwords_i) #말뭉치에서 불용어 제거

  #말뭉치들을 다시 처음처럼 문장으로 합침
  fin_sen=" ".join(corpus)
  fin_xdata.append(fin_sen)


#x_data를 불용어를 제거한 fin_xdata로 갱신
x_data=fin_xdata

# spam, ham 라벨을 대응하는 index로 치환하기위한 딕셔너리
label2index_dict = {'spam':0, 'ham':1}

# 문자열이었는데 숫자로 바꾼 데이터를 넣을 거임
# indexing 한 데이터를 넣을 리스트 선언
# 초기화
indexing_x_data, indexing_y_data = [], []

# y_data에 있는 각 문장의 단어들을 대응하는 index로 치환하고 그 결과값을 indexing_y_data에 저장
for label in y_data:
  indexing_y_data.append(label2index_dict[label])

# x_data를 사용하여 딕셔너리 생성
tokenizer.fit_on_texts(x_data)                   

# x_data에 있는 각 문장의 단어들을 대응하는 index로 치환하고 그 결과값을 indexing_x_data에 저장
indexing_x_data = tokenizer.texts_to_sequences(x_data)    

print("x_data indexing 하기 전 : " + str(x_data[0]))
print("x_data indexing 하기 후 : " + str(indexing_x_data[0]))
print("y_data indexing 하기 전 : " + str(y_data[0]))
print("y_data indexing 하기 후 : " + str(indexing_y_data[0]))

##train 할 데이터 다 만듦

영어 불용어 개수 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


<h2>SVM 모델 학습</h2>

<pre>
<b>1. 데이터의 문장 길이를 고정된 길이로 변환</b>
  예제 코드에서는 60으로 맞추어 변환
  
  문장 길이가 60 초과인 경우 뒷 부분 제거
  문장 길이가 60 미만인 경우 나머지 부분을 0으로 채움
  
  예시)
    문장 길이를 5로 맞추고자 할 경우
    
    문장 길이가 5보다 큰 경우
    문장 : 나는 어제 집에서 저녁으로 김치찌개를 먹었다, indexing_문장 : 38, 93, 239, 240, 241, 242
    38, 93, 239, 240, 241, 242 -> 38, 93, 239, 240, 241
    
    문장 길이가 5보다 작은 경우
    문장 : 나는 김치찌개를 좋아해, indexing_문장 : 74, 248, 127
    74, 248, 127 -> 74, 248, 127, 0, 0
    
<b>2. 입력 데이터를 9 대 1 비율로 나누어 학습, 평가에 사용</b>
  train_x = [ 문장1, 문장2, 문장3, ... 문장90]
  train_y = [ 문장1의 라벨, 문장2의 라벨, 문장3의 라벨, ... 문장90의 라벨]
  test_x = [ 문장91, 문장92, 문장93, ... 문장100]
  test_y = [ 문장91의 라벨, 문장92의 라벨, 문장93의 라벨, ... 문장100의 라벨]
  
<b>3. svm.fit(x, y) 함수를 사용하여 SVM 모델 학습</b>
  svm.fit(x, y)
  args
    x : indexing 된 문장들이 있는 리스트
    y : x의 각 문장에 대응하는 라벨이 있는 리스트
  return : 
    X
</pre>

In [4]:
# svm중 SVC(ham이냐 spam이냐 분류함)만 가져올 거다
from sklearn.svm import SVC

# svm에서는 단어 하나하나가 차원축임
# a라는 단어가 3개, b라는 단어가 2개면 (3,2)
# list니까 1차원?? 아니다

# 문장의 길이를 max_length으로 맞춰 변환
max_length = 60
for index in range(len(indexing_x_data)):
  length = len(indexing_x_data[index])
  
  if(length > max_length):
    indexing_x_data[index] = indexing_x_data[index][:max_length]
  elif(length < max_length):
    indexing_x_data[index] = indexing_x_data[index] + [0]*(max_length-length) # 빈 공간은 0으로 채워서 길이를 맞춤
    
    
# 전체 데이터를 9:1의 비율로 나누어 학습 및 평가 데이터로 사용
# train set: 90%, test set: 10%
number_of_train = int(len(indexing_x_data)*0.9)

train_x = indexing_x_data[:number_of_train]
train_y = indexing_y_data[:number_of_train]
test_x = indexing_x_data[number_of_train:] # 90번째부터 끝까지
test_y = indexing_y_data[number_of_train:]

print("train_x의 개수 : " + str(len(train_x)))
print("train_y의 개수 : " + str(len(train_y)))
print("test_x의 개수 : " + str(len(test_x)))
print("test_y의 개수 : " + str(len(test_y)))

# !중요!
# 단어 하나하나가 차원, feature이 되는데 차원이 굉장히 많은 문제는 linear kernel 이 working 잘함
# 차원이 작으면 polynomial 가 좋음
# C는 soft margin 값 (어느정도 마진까지 허락할지)
# 실제로는 clear 하게 나누는 선은 못구하므로 어느정도 에러는 허용해야 함
# 마진안에 틀린 것들이 있을텐데 얼마나 허락해줄지
# hard margin: 하나도 에러 허용안해줌, soft margin: 어느정도는 에러를 허용해주겠다 -> 대부분은 soft margin
svm = SVC(kernel='linear', C=1e10) # linear 커널 사용
svm.fit(train_x, train_y)

train_x의 개수 : 90
train_y의 개수 : 90
test_x의 개수 : 10
test_y의 개수 : 10


SVC(C=10000000000.0, kernel='linear')

<h2>SVM 모델을 이용한 평가</h2>

<pre>
<b>1. svm.predict(data) 함수를 사용하여 SVM 모델을 이용하여 평가</b>
  
  svm.predict(data)
  args
    data : indexing 된 문장들이 있는 리스트
  return : 
    입력 문장들에 대한 모델의 출력 라벨 리스트
    
<b>2. 성능 측정</b>
  정답 라벨과 모델의 출력 라벨을 비교하여 성능 측정
  
<b>3. tokenizer.sequences_to_texts(data) 함수를 이용하여 indexing 된 데이터를 단어로 치환</b>

  tokenizer.sequences_to_texts(data)
  args
    data : indexing 된 리스트
  return : 
    단어로 치환된 리스트
    
  예시)
    [38, 93, 239, 240, 241, 242, 53, 11, 243, 72, 94, 244, 245, 126, 246, 247, 73, 74, 248, 127] -> Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    
<b>4. 입력 문장에 대한 모델의 출력과 정답 출력</b>

</pre>

In [5]:
predict = svm.predict(test_x) 

#predict 한 결과 출력
correct_count = 0
for index in range(len(predict)):
  #정답과 predict 한 게 맞냐, 몇개 맞았는지 count
  if(test_y[index] == predict[index]):
    correct_count += 1
    
## 이거 확률 구하는 거 시험에 나올 듯??
# acuuracy 측정    
accuracy = 100.0*correct_count/len(test_y)
# precision은 ham이라고 내뱉은 것중에 맞은 것
# recall은 원래 ham 정답인 것 중에 맞은 것
# 원래 accuracy만 하면 안됨
# 왜 precision, recall 구해야함?, label이 언발란스하기땜에 클래스별로 해줘야함
# 클래스별로 fr이 나와야하고 그거의 평균이 매크로평균?? 


print("Accuracy: " + str(accuracy))

index2label = {0:"spam", 1:"ham"}

# 숫자를 언어로 다시 바꿈
test_x_word = tokenizer.sequences_to_texts(test_x)

for index in range(len(test_x_word)):
  print()
  print("문장 : ", test_x_word[index])
  print("정답 : ", index2label[test_y[index]])
  print("모델 출력 : ", index2label[predict[index]])

Accuracy: 60.0

문장 :  yeah do don‘t stand close tho you‘ll catch something
정답 :  ham
모델 출력 :  ham

문장 :  sorry pain ok meet another night spent late afternoon casualty means done stuff42moro includes time sheets that sorry
정답 :  ham
모델 출력 :  ham

문장 :  smile pleasure smile pain smile trouble pours like rain smile sum1 hurts u smile becoz someone still loves see u smiling
정답 :  ham
모델 출력 :  spam

문장 :  please call customer service representative 0800 169 6031 10am 9pm guaranteed ￡1000 cash ￡5000 prize
정답 :  spam
모델 출력 :  ham

문장 :  havent planning buy later check already lido got 530 show e afternoon u finish work already
정답 :  ham
모델 출력 :  spam

문장 :  free ringtone waiting collected simply text password mix 85069 verify get usher britney fml po box 5249 mk17 92h 450ppw 16
정답 :  spam
모델 출력 :  spam

문장 :  watching telugu movie wat abt u
정답 :  ham
모델 출력 :  ham

문장 :  see finish loads loans pay
정답 :  ham
모델 출력 :  ham

문장 :  hi wk ok hols now yes bit run forgot hairdressers appointment four

nltk.download("stopwords") 를 통해 불용어를 다운 받고 불용어를 제거한 말뭉치들을 모을 리스트 corpus를 생성했다.
for 문을 통해 x_data에서 불용어를 다 제거했는데 이때 불용어 리스트를 출력하면 불용어들이 다 소문자로 출력이 돼 정확도를 높이기 위해 불용어를 제거하기 전 x_data 말뭉치들을 다 소문자화한 후 불용어를 제거했다.
그 결과 소문자화하기 전 정확도는 50%가 나왔는데 소문자화한 후 정확도는 60%로 올라갔다.